## Setup

Begin by installing and importing some necessary libraries

# The way this tutorial uses the `TimeDistributed` layer requires TF>=2.10
pip install -U "tensorflow>=2.10.0"

pip install remotezip tqdm opencv-python
pip install -q git+https://github.com/tensorflow/docs

pip install imageio

import imageio

pip install scikit-learn

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.nn.parameter import Parameter
from sklearn.preprocessing import LabelEncoder

import cv2 # process video files
import imageio
from tensorflow_docs.vis import embed #for embedding data in a Jupyter notebook
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import math

## Download dataset



In [5]:
dataset_file_path = "C:/Users/s2165/OneDrive/Documents/UM/Master Second Sem/WOA7015 ADVANCED MACHINE LEARNING/Group Project"

dataset_filepath = dataset_file_path+"/input_data_new.csv"
df = pd.read_csv(dataset_filepath)

In [6]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


## Create frames from each video file



Using pandas library to process dataframe and returen to the seconds.

In [7]:
def convert_time_to_seconds(time_str):
    h, m, s = map(int, time_str.split(':'))
    return h * 3600 + m * 60 + s

df["start_time_seconds"]=df["Start_Time"].apply(convert_time_to_seconds)
df["end_time_seconds"]=df["End_Time"].apply(convert_time_to_seconds)
class_list = df.Movement_Name.unique()

In [8]:
#Extract frame from video
def extract_frame(path,start_time,end_time):
  frame_list = []
  SEQUENCE_LENGTH = 24

  # Open the video file for reading
  video_reader = cv2.VideoCapture(path)

  # Get the frames per second (fps), start_frame, and end_frame for the specified video duration
  fps = int(video_reader.get(cv2.CAP_PROP_FPS))
  frame_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
  start_frame = int(start_time*fps)
  end_frame = int(end_time*fps)
  interval_frame = end_frame - start_frame

  #calculate the skip frame number formula total duration of interval frame / sequence length
  skip_frame_window = max(interval_frame/SEQUENCE_LENGTH,1)

  # Get the height and width of the video frames
  height = video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT)
  width = video_reader.get(cv2.CAP_PROP_FRAME_WIDTH)
  print(f'fps:{fps}, frame_cnt:{frame_count} interval_frame:{interval_frame}, height:{height}, width:{width}, start_frame:{start_frame},end_frame:{end_frame},skip:{skip_frame_window}')


  # Set the video reader to the start frame
  video_reader.set(cv2.CAP_PROP_POS_FRAMES,start_frame)
  current_frame = start_frame

  # Iterate through frames in the specified interval
  while video_reader.isOpened() and math.ceil(current_frame) < end_frame:
    ret, frame = video_reader.read()
    if not ret:
      break

    # Convert the frame from BGR to RGB
    frame_rgb_opencv = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Resize the frame to a specific size (720x1280)
    resized_frame = cv2.resize(frame_rgb_opencv,(128,128))

    #normalize data to [0,1]
    frame = tf.image.convert_image_dtype(resized_frame, tf.float32)
      
    # Append the resized frame to the frame_list
    frame_list.append(frame)

    # Set the video reader to the next frame position
    video_reader.set(cv2.CAP_PROP_POS_FRAMES, int(current_frame + skip_frame_window) )

    # Increment the current_frame by skip_frame_window
    current_frame += skip_frame_window

    #current_frame += 1

  # Release the video reader and close any remaining OpenCV windows
  video_reader.release()
  cv2.destroyAllWindows()
  print(len(frame_list))
    
  # Return the list of extracted frames
    
  return frame_list;

In [9]:
X = [] #Save the frame
Y = [] #Save the lable

for index,row in df.iterrows():
  start_time = row["start_time_seconds"]
  end_time = row["end_time_seconds"]
  path = dataset_file_path + row["GD_Path"] # Get the vidoe file path
  label = row["Movement_Name"] # Get the Movement lable

  Y.append(np.where(class_list == label)[0][0])
  X.append(np.asarray(extract_frame(path,start_time,end_time)))

fps:0, frame_cnt:0 interval_frame:0, height:0.0, width:0.0, start_frame:0,end_frame:0,skip:1
0
fps:0, frame_cnt:0 interval_frame:0, height:0.0, width:0.0, start_frame:0,end_frame:0,skip:1
0
fps:0, frame_cnt:0 interval_frame:0, height:0.0, width:0.0, start_frame:0,end_frame:0,skip:1
0
fps:0, frame_cnt:0 interval_frame:0, height:0.0, width:0.0, start_frame:0,end_frame:0,skip:1
0
fps:0, frame_cnt:0 interval_frame:0, height:0.0, width:0.0, start_frame:0,end_frame:0,skip:1
0
fps:0, frame_cnt:0 interval_frame:0, height:0.0, width:0.0, start_frame:0,end_frame:0,skip:1
0
fps:0, frame_cnt:0 interval_frame:0, height:0.0, width:0.0, start_frame:0,end_frame:0,skip:1
0
fps:0, frame_cnt:0 interval_frame:0, height:0.0, width:0.0, start_frame:0,end_frame:0,skip:1
0
fps:0, frame_cnt:0 interval_frame:0, height:0.0, width:0.0, start_frame:0,end_frame:0,skip:1
0
fps:0, frame_cnt:0 interval_frame:0, height:0.0, width:0.0, start_frame:0,end_frame:0,skip:1
0
fps:0, frame_cnt:0 interval_frame:0, height:0.0, w

In [10]:
def to_gif(images):
  converted_images = np.clip(images*255, 0, 255).astype(np.uint8)
  imageio.mimsave('./animation.gif', converted_images, fps=10)
  # return embed.embed_file('./animation.gif')
  return HTML('<img src="./animation.gif">')

In [11]:
to_gif(X[0])

ValueError: Image data must be a sequence of ndimages.

In [ ]:
#encode the label by using one-hot encoded
one_hot_encoded_labels = to_categorical(Y)

# Split dataset into training and testing dataset

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,one_hot_encoded_labels,test_size=0.2,
                                                 shuffle=True,random_state=42)

In [ ]:
class_list.size